In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

In [2]:
df = pd.read_excel('var_apl.xlsx', sheet_name=2)
df = df.set_index('Date')

In [3]:
conf = 0.99
long = True
posiciones = np.array([1000*1245, 10000*871])

In [4]:
def VaR_APL(data: pd.DataFrame, posiciones: np.array, conf: float, long: bool):
    """ 
    Función que calcula el VaR y C-VaR de un portafolio de activos financieros ajustado por costos de liquidez.

    Args:
    data (pd.DataFrame): DataFrame con los bid y ask de los activos financieros.
    posiciones (np.array): Vector con las posiciones de los activos financieros.
    conf (float): Nivel de confianza.
    long (bool): True si el portafolio es largo, False si es corto.

    Returns:
    pd.DataFrame: DataFrame con el VaR y C-VaR ajustado por costos de liquidez.
    """

    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost

    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return resultados

In [5]:
VaR_APL(df, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0596,"7,268,538.2184"
1,VaR Ajustado Promedio,0.0635,"7,735,139.8749"
2,VaR Ajustado Estresado,0.0835,"10,179,127.4718"
3,C-VaR,0.0707,"8,613,290.5188"
4,C-VaR Ajustado Promedio,0.0745,"9,079,892.1753"
5,C-VaR Ajustado Estresado,0.0946,"11,523,879.7722"
